In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
WITH
u AS (SELECT *,
             CASE
                  WHEN lt_day <= 365 THEN 'новый'
                  WHEN lt_day > 365 THEN 'старый'
             END AS is_new,
             CASE 
                  WHEN nps_score >= 0 AND nps_score <= 6 THEN 'критик'
                  WHEN nps_score >= 7 AND nps_score <= 8 THEN 'нейтрал'
                  WHEN nps_score >= 9 AND nps_score <= 10 THEN 'сторонник'
             END AS nps_group,
             CASE
                  WHEN gender_segment = 1 THEN 'жен'
                  WHEN gender_segment = 0 THEN 'муж'
                  ELSE 'неопределённый'
             END as gender
     FROM user),      
loc AS (SELECT *
        FROM location),
a AS (SELECT age_gr_id,
             title
      FROM age_segment),
t AS (SELECT tr_gr_id,
             title
      FROM traffic_segment),
l AS (SELECT lt_gr_id,
             title
      FROM lifetime_segment)
SELECT u.user_id,
       u.lt_day,
       u.is_new,
       u.age,
       u.gender,
       u.os_name,
       u.cpe_type_name,
       u.nps_score,
       u.nps_group,
       loc.country,
       loc.city,
       SUBSTRING(a.title, 4) AS age_segment,
       SUBSTRING(t.title, 4) AS traffic_segment,
       SUBSTRING(l.title, 4) AS lifetime_segment
FROM u LEFT JOIN loc ON u.location_id=loc.location_id
LEFT JOIN a ON u.age_gr_id=a.age_gr_id
LEFT JOIN t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN l ON u.lt_gr_id=l.lt_gr_id;            
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,nps_score,nps_group,country,city,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,старый,45.0,жен,ANDROID,SMARTPHONE,10,сторонник,Россия,Уфа,45-54,1-5,36+
1,A001WF,2344,старый,53.0,муж,ANDROID,SMARTPHONE,10,сторонник,Россия,Киров,45-54,1-5,36+
2,A003Q7,467,старый,57.0,муж,ANDROID,SMARTPHONE,10,сторонник,Россия,Москва,55-64,20-25,13-24


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Дашборды:**

https://public.tableau.com/views/Project_telecommunication_Khudyakov/Project_telecommunication_khudyakov?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link